<div style="border:3px solid black; padding:20px">
    
    
## * 진동데이터 활용 충돌체 탐지 AI 경진대회

### + 배경
+ 원자력발전소 냉각재계통 내부에 존재하는 충돌체는 기기의 손상을 유발할 수 있습니다.<br>원전 현장에서는 기기의 이상징후를 조기에 진단하여 사고를 방지하고자 합니다.<br>본 대회에서는 기기의 이상징후를 조기에 진단할 수 인공지능 기술 연구 활성화를 목적으로 합니다.

### + 기간
+ 2020.06.01 ~ 2020.07.10


+ 대회 링크: https://dacon.io/competitions/official/235614/overview/
</div>

## < 필요 라이브러리 Import >

In [ ]:
import os
import json
import numpy as np

import matplotlib.pyplot as plt; %matplotlib inline
import random
import pandas as pd

import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K

from tqdm import tqdm
from sklearn.model_selection import KFold

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D
from tensorflow.keras.layers import Input, Concatenate, GlobalAveragePooling2D, ZeroPadding2D, SeparableConv2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model

from tensorflow.keras.utils import get_custom_objects

np.random.seed(777)
random.seed(777)
tf.random.set_seed(777)

## < Data Load and Preprocessing >
+ 각 진동체가 움직이는 범위의 평균과 표준편차를 특성으로 만들고,
+ 전체 진동체의 평균과 표준편차를 특성으로 만들어, X_data에 concat합니다.
+ 푸리에 변환을 활용한 특성 추가, 다양한 평균 및 표준편차를 활용한 특성을 추가해보았지만 여기서 사용한 특성으로 가장 좋은 성능을 얻을 수 있었습니다.

In [ ]:
X_data = pd.read_csv('./data/train_features.csv')
X_data_test = pd.read_csv('./data/test_features.csv')

for col in ['S1', 'S2', 'S3', 'S4']:
    col_mean = X_data[col].mean()
    col_std = X_data[col].std()

    X_data[col] = (X_data[col] - col_mean) / (col_std + 1e-10)
    X_data_test[col] = (X_data_test[col] - col_mean) / (col_std + 1e-10)

In [ ]:
# 학습용 데이터 전처리를 수행합니다.
X_data = X_data.to_numpy()
X_data = X_data[:,1:]
X_data = X_data.reshape((2800, 375, 5))

trainv_mean = np.mean(X_data[:, :, 1:-1], axis = 2)[..., np.newaxis]
trainv_std = np.std(X_data[:, :, 1:-1], axis = 2)[..., np.newaxis]

X_data = np.concatenate((X_data, trainv_mean),axis = -1)
X_data = np.concatenate((X_data, trainv_std),axis = -1)
X_data = X_data.reshape((2800, 375, 7, 1))
print(X_data.shape)
    
Y_data = np.loadtxt('./data/train_target.csv',skiprows=1,delimiter=',')
Y_data = Y_data[:,1:]
print(Y_data.shape)

# 테스트용 데이터 전처리를 수행합니다.
X_data_test = X_data_test.to_numpy()
X_data_test = X_data_test[:,1:]
X_data_test = X_data_test.reshape((700, 375, 5))

testv_mean = np.mean(X_data_test[:, :, 1:-1], axis = 2)[..., np.newaxis]
testv_std = np.std(X_data_test[:, :, 1:-1], axis = 2)[..., np.newaxis]

X_data_test = np.concatenate((X_data_test, testv_mean),axis = -1)
X_data_test = np.concatenate((X_data_test, testv_std),axis = -1)
X_data_test = X_data_test.reshape((700, 375, 7, 1))
print(X_data_test.shape)

In [ ]:
X_train = X_data; Y_train = Y_data

## < Loss Define >

In [ ]:
weight1 = np.array([1,1,0,0]) # x, y
weight2 = np.array([0,0,1,1]) # M, V

def my_loss(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    
    return K.mean(K.square(divResult))

def my_loss_E1(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred)*weight1)/2e+04

def my_loss_E2(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    
    return K.mean(K.square(divResult)*weight2)

## < Model Define >
+ Mish 활성화 함수를 사용했습니다.
+ CNN을 통해 각 진동체의 특징을 활용하도록 합니다. 하지만 데이터의 형태가 (Batch_size, row, col, channel)로 구성되어 있기 떄문에
+ CNN 2D를 사용하는 대신 (3, 1), (5, 1)과 같은 필터 크기를 사용토록 합니다.
+ 아래 모델에서는 [3, 5, 7, 9, 11, 13, 15] 크기가 사용되었습니다.
+ x, y(충격 좌표)는 함께 학습하고, M(질량)과 V(충돌 속도)는 따로 분리하여 학습하도록 합니다. 이는 weight2 변수를 통해 Loss에서 통제합니다.

In [ ]:
def Mish(x):
    return x * K.tanh(K.softplus(x))

get_custom_objects().update({'mish': Mish})

def conv_block(inputs, nf, fs, pool_size = (2, 1), padding = 'valid', activation = 'elu', downsample = True):
    x = BatchNormalization()(inputs)
    x = Conv2D(filters = nf, kernel_size = fs, padding = padding)(x)
    x = Activation(activation)(x)
    
    if downsample:
        x = MaxPooling2D(pool_size = pool_size)(x)
    
    return x

In [ ]:
def get_model(train_target):
    '''
        #train_target: 0이면 x,y, 1이면 M, 2이면 V
    '''
    
    activation = 'mish'
    padding = 'valid'
    nf = 16
    fs_3 = (3, 1); fs_5 = (5, 1)
    fs_7 = (7, 1); fs_9 = (9, 1)
    fs_11 = (11, 1); fs_13 = (13, 1)
    fs_15 = (15, 1)
    
    inputs = Input(shape = (375, 7, 1))

    init_layer = conv_block(inputs, nf, (3, 1), padding = padding, activation = activation)
    init_layer = conv_block(init_layer, nf * 2, (3, 1), padding = padding, activation = activation)
    
    x_3 = conv_block(init_layer, nf*16, (1, 1), pool_size = (2, 1), padding = padding, activation = activation, downsample = False)
    x_3 = conv_block(x_3, nf*4, fs_3, pool_size = (2, 1), padding = padding, activation = activation)
    x_3 = conv_block(x_3, nf*8, fs_3, pool_size = (2, 1), padding = padding, activation = activation) # (42, 5, 256)
    x_3 = conv_block(x_3, nf*16, fs_5, pool_size = (2, 1), padding = padding, activation = activation, downsample = False) # (42, 5, 256)
    
    x_5 = conv_block(init_layer, nf*16, (1, 1), pool_size = (2, 1), padding = padding, activation = activation, downsample = False)
    x_5 = conv_block(x_5, nf*4, fs_5, pool_size = (2, 1), padding = padding, activation = activation)
    x_5 = conv_block(x_5, nf*8, fs_5, pool_size = (2, 1), padding = padding, activation = activation)
    x_5 = conv_block(x_5, nf*16, fs_5, pool_size = (2, 1), padding = padding, activation = activation, downsample = False) # (44, 5, 128)
    x_5 = ZeroPadding2D(padding = ((1, 0), 0))(x_5)

    x_7 = conv_block(init_layer, nf*16, (1, 1), pool_size = (2, 1), padding = padding, activation = activation, downsample = False)
    x_7 = conv_block(x_7, nf*4, fs_7, pool_size = (2, 1), padding = padding, activation = activation)
    x_7 = conv_block(x_7, nf*8, fs_7, pool_size = (2, 1), padding = padding, activation = activation)
    x_7 = conv_block(x_7, nf*16, fs_7, pool_size = (2, 1), padding = padding, activation = activation, downsample = False) # (44, 5, 128)
    x_7 = ZeroPadding2D(padding = ((5, 0), 0))(x_7)

    x_9 = conv_block(init_layer, nf*16, (1, 1), pool_size = (2, 1), padding = padding, activation = activation, downsample = False)
    x_9 = conv_block(x_9, nf*8, fs_9, pool_size = (2, 1), padding = padding, activation = activation)
    x_9 = conv_block(x_9, nf*16, fs_9, pool_size = (2, 1), padding = padding, activation = activation)

    x_11 = conv_block(init_layer, nf*16, (1, 1), pool_size = (2, 1), padding = padding, activation = activation, downsample = False)
    x_11 = conv_block(x_11, nf*8, fs_11, pool_size = (2, 1), padding = padding, activation = activation)
    x_11 = conv_block(x_11, nf*16, fs_11, pool_size = (2, 1), padding = padding, activation = activation)
    x_11 = ZeroPadding2D(padding = (1, 0))(x_11)

    x_13 = conv_block(init_layer, nf*16, (1, 1), pool_size = (2, 1), padding = padding, activation = activation, downsample = False)
    x_13 = conv_block(x_13, nf*8, fs_13, pool_size = (2, 1), padding = padding, activation = activation)
    x_13 = conv_block(x_13, nf*16, fs_13, pool_size = (2, 1), padding = padding, activation = activation)
    x_13 = ZeroPadding2D(padding = ((3, 0), 0))(x_13)
    
    x_15 = conv_block(init_layer, nf*16, (1, 1), pool_size = (2, 1), padding = padding, activation = activation, downsample = False)
    x_15 = conv_block(x_15, nf*8, fs_15, pool_size = (2, 1), padding = padding, activation = activation)
    x_15 = conv_block(x_15, nf*16, fs_15, pool_size = (2, 1), padding = padding, activation = activation)
    x_15 = ZeroPadding2D(padding = ((5, 0), 0))(x_15)

    concat_layer = Concatenate()([x_3, x_5, x_7, x_9, x_11, x_13, x_15])
    x = conv_block(concat_layer, nf*32, (1, 1), padding = padding, activation = activation, downsample = False)
    x = conv_block(x, nf*16, fs_3)
    x = conv_block(x, nf*32, fs_3)
    
    x = Flatten()(x)
    
    x = Dense(128)(x)
    x = Activation(activation)(x)
    x = Dense(64)(x)
    x = Activation(activation)(x)
    x = Dense(32)(x)
    x = Activation(activation)(x)
    x = Dense(16)(x)
    x = Activation(activation)(x)
    outputs = Dense(4)(x)
    
    model = Model(inputs = inputs, outputs = outputs)

    optimizer = tensorflow.keras.optimizers.Adam()
    
    # M과 V는 따로 학습시킬 수 있도록 합니다.
    global weight2
    if train_target == 1: # only for M
        weight2 = np.array([0,0,1,0])
    elif train_target == 2: # only for V
        weight2 = np.array([0,0,0,1])
       
    if train_target==0:
        model.compile(loss=my_loss_E1,
                  optimizer=optimizer,
                 )
    else:
        model.compile(loss=my_loss_E2,
                  optimizer=optimizer,
                 )

    return model

## < Training >

In [ ]:
def train(model,X,Y, X_val, Y_val, train_target, fold_num, epochs = 200):
    # 폴더가 존재하지 않으면 생성합니다.
    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)
    
    # target에 따라 다른 모델을 저장합니다.
    if train_target == 0:
        model_path = MODEL_SAVE_FOLDER_PATH + f'xymodel{fold_num}.hdf5'
    elif(train_target == 1):
        model_path = MODEL_SAVE_FOLDER_PATH + f'mmodel{fold_num}.hdf5'
    else:
        model_path = MODEL_SAVE_FOLDER_PATH + f'vmodel{fold_num}.hdf5'

    best_save = ModelCheckpoint(model_path, save_best_only=True, monitor='val_loss', mode='min')
    callbacks = [best_save, ReduceLROnPlateau(monitor='val_loss', factor = 0.1, min_lr = 1e-6, patience = 20, verbose = 1)]

    history = model.fit(X, Y,
                        epochs=epochs, batch_size=256,
                        shuffle=True, verbose = 2,
                        validation_data = (X_val, Y_val),
                        callbacks=[callbacks])

    return model

In [ ]:
# 테스트를 위한 지표 정의
def E1(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: distance error normalized with 2e+04
    '''
    
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)


def E2(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    
    
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))

def load_best_model(train_target, fold_num):

    if train_target == 0:
        model = load_model(f'./model/xymodel{fold_num}.hdf5' , custom_objects={'my_loss_E1': my_loss, 'Mish':Mish})
    elif(train_target == 1):
        model = load_model(f'./model/mmodel{fold_num}.hdf5', custom_objects = {'my_loss_E2':my_loss, 'Mish':Mish})
    else:
        model = load_model(f'./model/vmodel{fold_num}.hdf5' , custom_objects={'my_loss_E2': my_loss, 'Mish':Mish})

    score = model.evaluate(X_data, Y_data, verbose=0)
    print('loss:', score)

    pred = model.predict(X_data)

    # show only one sample
    i=0
    print('정답(original):', Y_data[i])
    print('예측값(original):', pred[i])

    print(E1(pred, Y_data))
    print(E2(pred, Y_data))
#     print(E2M(pred, Y_data))
#     print(E2V(pred, Y_data))    
    
#     if train_target ==0:
#         plot_error(4,pred,Y_data)
#     elif train_target ==1:
#         plot_error(2,pred,Y_data)
#     elif train_target ==2:
#         plot_error(3,pred,Y_data)    
    
    return model

+ 주로 100 ~ 150 epoch 사이에서 수렴을 보이기 때문에 최대 200 epoch으로 설정했습니다.

In [ ]:
preds = []
epochs = 200

kfold = KFold(n_splits = 5, random_state = 777)

for i, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
    x_train_fold, x_val_fold = X_train[train_idx], X_train[val_idx]y_train_fold, y_val_fold = Y_train[train_idx], Y_train[val_idx]

    pred = []
    submit = pd.read_csv('./data/sample_submission.csv')
    print(f'fold {i} start!!')

    for train_target in range(3):
        model = get_model(train_target)
        train(model, x_train_fold, y_train_fold, x_val_fold, y_val_fold, train_target, fold_num = i, epochs = epochs)
        best_model = load_best_model(train_target, fold_num = i)

        pred_data_test = best_model.predict(X_data_test)

        if train_target == 0: # x,y 학습
            submit.iloc[:,1] = pred_data_test[:,0]
            submit.iloc[:,2] = pred_data_test[:,1]
        elif train_target == 1: # M 학습
            submit.iloc[:,3] = pred_data_test[:,2]
        elif train_target == 2: # V 학습
            submit.iloc[:,4] = pred_data_test[:,3]

    preds.append(submit.to_numpy()[:, 1:])
    print(f'fold {i} End!!')
    
    K.clear_session()

## < Submission >

In [ ]:
preds = np.mean(preds, axis = 0)
submit = pd.read_csv('./data/sample_submission.csv')

submit.iloc[:,1] = preds[:, 0]
submit.iloc[:,2] = preds[:, 1]
submit.iloc[:,3] = preds[:, 2]
submit.iloc[:,4] = preds[:, 3] 

submit.to_csv('./submit_current.csv', index = False)